<a href="https://colab.research.google.com/github/RobinSmits/FakeNews-Generator-And-Detector/blob/main/FakeNews_Classifier_RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the first notebook we used a T5 model to train on the 'title' and 'description' of the 'ag_news_subset' dataset. As a second step we used that T5 model to generate a CSV with fake news based on the input 'title'.

In this second notebook we will use the real and fake news in the generated csv ('t5_generated_fake_news.csv') file to train a RoBERTa Base model to classify the real (column: 'description' - label: 0) or fake news (column: 'generated' - label: 1).

In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

# Install Specific Versions
!pip install tensorflow==2.4.1
!pip install tensorflow-datasets==4.1.0
!pip install transformers==4.2.2
!pip install sentencepiece==0.1.95

# Import Packages
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import *
import sentencepiece

     |████████████████████████████████| 3.6MB 6.6MB/s 
  Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1
     |████████████████████████████████| 1.8MB 6.8MB/s 
     |████████████████████████████████| 890kB 22.6MB/s 
     |████████████████████████████████| 2.9MB 36.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=63de4a97b6870b784ef4aff2d8845e645c46d698b38c000fd450275f98c1fb7e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 6.4MB/s 


I've created and tested these notebooks on Google Colab Pro and used Google Drive to store and load any files created. 

If you run the code locally on a computer then modify the 'WORK_DIR' accordingly. Google Drive will not be needed in that case.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set Folder to use...
WORK_DIR = '/content/drive/My Drive/fake_news/'
os.makedirs(WORK_DIR, exist_ok = True) 

Next we set some config for the device to use. We also set the necessary constants. 

For the learning rate you could try different settings. Setting the learning rate to high will very quickly overfit the model

And finally we set the RoBERTa tokenizer and config to use.

In [ ]:
# Configure Strategy. Assume TPU...if not set default for GPU/CPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy()

# Set Auto Tune
AUTO = tf.data.experimental.AUTOTUNE

# Supress Warnings
tf.autograph.set_verbosity(0, False)

# Constants
MAX_LEN = 512
EPOCHS = 3
VERBOSE = 1  

# Batch Size
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
print(f'Batch Size: {BATCH_SIZE}')

# Learning Rate
LR = 1.25e-6 * strategy.num_replicas_in_sync
print('Learning Rate: {}'.format(LR))

# Set RoBERTa Type
roberta_type = 'roberta-base'
print(f'RoBERTa Model Type: {roberta_type}')

# Set RoBERTa Config
roberta_config = RobertaConfig.from_pretrained(roberta_type, num_labels = 2) # Binary classification so set num_labels = 2

# Set RoBERTa Tokenizer
roberta_tokenizer = RobertaTokenizer.from_pretrained(roberta_type, 
                                                     add_prefix_space = False,
                                                     do_lower_case = False)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.7.83.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.7.83.66:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Batch Size: 64
Learning Rate: 1e-05
RoBERTa Model Type: roberta-base


Next we define a function to process the 't5_generated_fake_news.csv' which is loaded as a Pandas Dataframe. We loop through all rows and from each row we use the columns 'description' and 'generated' as input for the RoBERTa model.

The 'description' input will be labelled with 0. The 'generated' input will be labelled with 1. The 'title' which we used in the T5 model is not used.

In [ ]:
def create_dataset(df):
    number_of_samples = df.shape[0]
    total_samples = 2 * df.shape[0]

    # Placeholders input
    input_ids = np.zeros((total_samples, MAX_LEN), dtype = 'int32')
    input_masks = np.zeros((total_samples, MAX_LEN), dtype = 'int32')
    labels = np.zeros((total_samples, ), dtype = 'int32')

    for index, row in tqdm(zip(range(0, total_samples, 2), df.iterrows()), total = number_of_samples):
        
        # Get title and description as strings
        description = row[1]['description']
        generated = row[1]['generated']

        # Process Description - Set Label for real as 0
        input_encoded = roberta_tokenizer.encode_plus(description, add_special_tokens = True, max_length = MAX_LEN, truncation = True, padding = 'max_length')
        input_ids_sample = input_encoded['input_ids']
        input_ids[index,:] = input_ids_sample
        attention_mask_sample = input_encoded['attention_mask']
        input_masks[index,:] = attention_mask_sample
        labels[index] = 0

        # Process Generated - Set Label for fake as 1
        input_encoded = roberta_tokenizer.encode_plus(generated, add_special_tokens = True, max_length = MAX_LEN, truncation = True, padding = 'max_length')
        input_ids_sample = input_encoded['input_ids']
        input_ids[index+1,:] = input_ids_sample
        attention_mask_sample = input_encoded['attention_mask']
        input_masks[index+1,:] = attention_mask_sample
        labels[index+1] = 1

    # Create DatasetDictionary structure is also preserved.
    dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': input_ids, 'attention_mask': input_masks}, labels))

    # Return Dataset
    return dataset

We load the csv file. Split it into an 80/20 train and test set and call the previously defined function to generate our final Tensorflow Datasets.

In [ ]:
# Import Generated Fake News
df = pd.read_csv(WORK_DIR + 't5_generated_fake_news.csv')

# Split in Train and Validation
train_df, val_df = train_test_split(df, test_size = 0.2, random_state = 42, shuffle = True)

# Show Sizes
print(f'Train Shape: {train_df.shape}')
print(f'Validation Shape: {val_df.shape}')

# Create Train Dataset
train_dataset = create_dataset(train_df)
train_dataset = train_dataset.shuffle(2048)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.repeat(-1)
train_dataset = train_dataset.prefetch(128)

# Create Validation Dataset
validation_dataset = create_dataset(val_df)
validation_dataset = validation_dataset.batch(BATCH_SIZE)
validation_dataset = validation_dataset.repeat(-1)
validation_dataset = validation_dataset.prefetch(128)

# Steps
train_steps = (train_df.shape[0] * 2) // BATCH_SIZE
val_steps = (val_df.shape[0] * 2) // BATCH_SIZE
print(f'Train Steps: {train_steps}')
print(f'Val Steps: {val_steps}')

Train Shape: (48000, 4)
Validation Shape: (12000, 4)



Train Steps: 1500
Val Steps: 375


Define a function to create and compile the RoBERTa base model.

In [ ]:
def create_model():
    # Create Model
    with strategy.scope():      
        model = TFRobertaForSequenceClassification.from_pretrained(roberta_type, config = roberta_config)
        
        optimizer = tf.keras.optimizers.Adam(learning_rate = LR)
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
        metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

        model.compile(optimizer = optimizer, loss = loss, metrics = [metric])        
        
        return model

Create a callback to save the model weights to storage.

In [ ]:
def ModelCheckpoint(model_name):
    return tf.keras.callbacks.ModelCheckpoint(model_name, 
                                              monitor = 'val_loss', 
                                              verbose = 1, 
                                              save_best_only = True, 
                                              save_weights_only = True, 
                                              mode = 'min', 
                                              period = 1)

Finally create the model and perform the training and validation.

In [ ]:
  # Create Model
  model = create_model()

  # Summary
  model.summary()

  # Fit Model
  model.fit(train_dataset,
            steps_per_epoch = train_steps,
            validation_data = validation_dataset,
            validation_steps = val_steps,
            epochs = EPOCHS, 
            verbose = VERBOSE,
            callbacks = [ModelCheckpoint(WORK_DIR + 'roberta_base_model.h5')])

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124055040 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  592130    
Total params: 124,647,170
Trainable params: 124,647,170
Non-trainable params: 0
_________________________________________________________________


Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f61f3f082a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).WARNING:tensorflow:AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x7f61d21186c8>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f61f3f082a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1500/1500 [==============================] - ETA: 0s - loss: 0.1892 - accuracy: 0.9122

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1500/1500 [==============================] - 452s 241ms/step - loss: 0.1891 - accuracy: 0.9122 - val_loss: 0.0873 - val_accuracy: 0.9661

Epoch 00001: val_loss improved from inf to 0.08726, saving model to /content/drive/My Drive/fake_news/roberta_base_model.h5
Epoch 2/3
1500/1500 [==============================] - 355s 237ms/step - loss: 0.0383 - accuracy: 0.9859 - val_loss: 0.0876 - val_accuracy: 0.9712

Epoch 00002: val_loss did not improve from 0.08726
Epoch 3/3
1500/1500 [==============================] - 355s 236ms/step - loss: 0.0217 - accuracy: 0.9921 - val_loss: 0.1455 - val_accuracy: 0.9573

Epoch 00003: val_loss did not improve from 0.08726


After training the model should achieve around 97% accuracy on the validation set.

And interresting point I noticed during testing is that the 97% accuracy was achieved based on the fake news as generated by a trained T5-base model.

When I performed the initial tests with fake news generated by a trained T5-small model the classifier could achieve around 99% accuracy. This makes sense as you would expect a smaller NLP model to be able to generate less 'natural looking' fake news. The 97% classification accuracy achieved on the fake news generated by the T5-base model more or less proves that it generates better fake news.